Brownian Dynamics in IMP<a id="mainpage"></a>
========================

**Table of contents**

 - [Setup](#setup)
 - [Representing parts](#repparts)
 - [Representing interactions](#repinteract)
 - [Representing dynamics](#repdynamics)
 - [Running the script](#running)
 - [Visualization](#visualization)
 - [Next steps](#nextsteps)


In this tutorial, we show how to setup and run a Brownian Dynamics simulation
in IMP.

# Setup<a id="setup"></a>


First, this cell will install IMP in the Google Colaboratory environment:

In [ ]:
!add-apt-repository -y ppa:salilab/ppa
!apt install imp
import sys, os, glob
sys.path.append(os.path.dirname(glob.glob('/usr/lib/python*/dist-packages/IMP')[0]))

Next, download the files for this tutorial from GitHub:

In [ ]:
!git clone https://github.com/salilab/imp_brownian_dynamics_tutorial.git
%cd imp_brownian_dynamics_tutorial/scripts/

We need to import IMP modules that we plan to use in the script:

In [ ]:
from __future__ import print_function, division
import IMP.atom
import IMP.algebra
import IMP.rmf
import IMP.core
import RMF
import IMP.container
import IMP.display
import GranuleFactory

Next, we can set parameters for the simulation:

In [ ]:
RMF_FILENAME = "pbc_simple.rmf"
# I. Parts parameters:
L = 50000 # Length of our bounding box, A
R = 20000 # PBC radius, A
R_NUCLEUS = 10000 # NE radius, A
N_GRANULES = 50 # Number of granules
R_GRANULES = 1500 # Radius of granules, A
N_GRANULE_PATCHES = 6

# II. Interaction parameters:
K_BB = 0.1  # Strength of the harmonic boundary box in kcal/mol/A^2
K_EXCLUDED=0.1 # Strength of lower-harmonic excluded volume score in kcal/mol/A^2

# III. Time parameters:
BD_STEP_SIZE_SEC= 10E-8
SIM_TIME_SEC= 0.050
bd_step_size_fs= BD_STEP_SIZE_SEC * 1E+15
sim_time_ns= SIM_TIME_SEC * 1E+9
RMF_DUMP_INTERVAL_NS= sim_time_ns / 1000.0

# Representing parts<a id="repparts"></a>

To represent parts of the system, we use a number of Python classes provided by IMP:

 - [IMP.Model](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1Model.html)
   A container for all of the system’s parts

 - [IMP.Particle](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1Particle.html)
   A particle is the elementary data unit describing a system part in IMP

 - [IMP.Decorator](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1Decorator.html)
   Dynamic descriptors of the properties of IMP particles

   - [IMP.core.XYZR](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1core_1_1XYZR.html)
   - [IMP.atom.Hierarchy](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1atom_1_1Hierarchy.html)
   - [IMP.atom.Diffusion](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1atom_1_1Diffusion.html)
   
 - [IMP.container](https://integrativemodeling.org/2.23.0/doc/ref/namespaceIMP_1_1container.html)
   Static or dynamic collections of particles, particle pairs, etc.

To see these classes used in a simulation, look at the file `pbc_simple.py` in the `scripts` directory. First, we create an IMP [Model](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1Model.html), and set up the root of a [hierarchy of particles](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1atom_1_1Hierarchy.html):

In [ ]:
# Model:
m = IMP.Model()
# Root of parts hierarchy:
p_root= IMP.Particle(m, "root")
h_root = IMP.atom.Hierarchy.setup_particle(p_root)

Next, we can set up a coarse-grained nucleus by calling the `create_nucleus` function. The nucleus particle is given XYZ coordinates and a radius using the [XYZR decorator](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1core_1_1XYZR.html). We then [add it](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1atom_1_1Hierarchy.html#a6a8c2cba388bf79adef66c28a7c78a57) to the existing hierarchy:

In [ ]:
def create_nucleus(m, R):
    '''
    Generate a coarse-grained spherical nuclear envelope
    of radius R in model m
    '''
    p = IMP.Particle(m, "nucleus")
    xyzr = IMP.core.XYZR.setup_particle(p)
    xyzr.set_coordinates_are_optimized(True)
    xyzr.set_coordinates([0,0,0])
    xyzr.set_radius(R)
    IMP.atom.Mass.setup_particle(p, 1.0) # fake mass
    IMP.display.Colored.setup_particle(p,
                                       IMP.display.get_display_color(2))
    IMP.atom.Hierarchy.setup_particle(p)
    return p

# Nucleus:
p_nucleus = create_nucleus(m, R_NUCLEUS)
h_nucleus = IMP.atom.Hierarchy(p_nucleus)
h_root.add_child(h_nucleus)

Next, we create a number of insulin granules at random locations in the
cytoplasm. This is done using a simple piece of code, that creates similar
XYZR particles, in `GranuleFactory.py`:

In [ ]:
# Granules hierarchy root:
p_granules_root= IMP.Particle(m, "Granules")
IMP.atom.Mass.setup_particle(p_granules_root, 1.0) # fake mass
h_granules_root= IMP.atom.Hierarchy.setup_particle(p_granules_root)
h_root.add_child(h_granules_root)

# PBC cytoplasm bounding sphere:
pbc_sphere= IMP.algebra.Sphere3D([0,0,0], R)
# Actual granules:
nucleus_sphere= IMP.core.XYZR(p_nucleus).get_sphere()
gf=GranuleFactory.GranuleFactory(
    model=m, default_R=R_GRANULES,
    cell_sphere=pbc_sphere,
    nucleus_sphere=nucleus_sphere)
for i in range(N_GRANULES):
    granule= gf.create_simple_granule("Granule_{}".format(i))
    h_granule= IMP.atom.Hierarchy(granule)
    h_granules_root.add_child(h_granule)

# Representing interactions<a id="repinteract"></a>

Next, we need to tell IMP the nature of the interactions in the system. These are handled by several IMP classes:

 - [Scores](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1SingletonScore.html) evaluate some property of the system, for example [how far a particle is outside of a sphere](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1core_1_1GenericBoundingSphere3DSingletonScore.html).

 - [Restraints](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1Restraint.html) apply Scores to a subset of the system.

 - [Containers](https://integrativemodeling.org/2.23.0/doc/ref/namespaceIMP_1_1container.html) describe the subsets that the Restraints
   use.

In [ ]:
# ----- II. System interactions: -----

# Outer bounding box for simulation:
bb = IMP.algebra.BoundingBox3D(IMP.algebra.Vector3D(-L/2, -L/2, -L/2),
                               IMP.algebra.Vector3D(L/2, L/2, L/2))

# Add enclosing spheres for pbc and outer simulation box
bb_harmonic= IMP.core.HarmonicUpperBound(0, K_BB)
pbc_bsss = IMP.core.BoundingSphere3DSingletonScore(bb_harmonic,
                                                   pbc_sphere)
outer_bbss = IMP.core.BoundingBox3DSingletonScore(bb_harmonic,
                                                  bb)
# Restraints - match score with particles:
rs = []
rs.append(IMP.container.SingletonsRestraint(pbc_bsss,
                                            h_granules_root.get_children()))
rs.append(IMP.container.SingletonsRestraint(outer_bbss,
                                            h_granules_root.get_children()))
# Add excluded volume restraints among all (close pairs of) particles:
ev = IMP.core.ExcludedVolumeRestraint(IMP.atom.get_leaves(h_root),
                                      K_EXCLUDED,
                                      10, # slack affects speed only
                                          # (slack of close pairs finder)
                                      "EV")
rs.append(ev)
# Scoring Function from restraints
sf = IMP.core.RestraintsScoringFunction(rs, "SF")

# Representing dynamics<a id="repdynamics"></a>

Finally we can tell IMP to run Brownian Dynamics. This uses the [IMP.atom.BrownianDynamics](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1atom_1_1BrownianDynamics.html) class. First, we set properties of the simulation, such as the temperature and number of time steps:

In [ ]:
# -------- III. System dynamics: --------

bd = IMP.atom.BrownianDynamics(m)
bd.set_log_level(IMP.SILENT)
bd.set_scoring_function(sf)
bd.set_maximum_time_step(bd_step_size_fs) # in femtoseconds
bd.set_temperature(300)

Next, we request output of the trajectory in RMF format (using the `add_optimizer_state` method):

In [ ]:
# -------- Add RMF visualization --------
def convert_time_ns_to_frames(time_ns, step_size_fs):
    '''
    Given time in nanoseconds time_ns and step size in femtosecond
    step_size_fs, return an integer number of frames greater or equal
    to 1, such that time_ns*step_size_fs is as close as possible to
    time_ns.
    '''
    FS_PER_NS= 1E6
    time_fs= time_ns * FS_PER_NS
    n_frames_float= (time_fs+0.0) / step_size_fs
    n_frames= int(round(n_frames_float))
    return max(n_frames, 1)
sim_time_frames= convert_time_ns_to_frames(sim_time_ns, bd_step_size_fs)
rmf_dump_interval_frames= convert_time_ns_to_frames(RMF_DUMP_INTERVAL_NS, bd_step_size_fs)

print("Simulation time {:.1e} ns / {} frames; "
      "RMF dump interval {:.1e} ns / {} frames".format(sim_time_ns,
                                                      sim_time_frames,
                                                       RMF_DUMP_INTERVAL_NS,
                                                      rmf_dump_interval_frames))

rmf = RMF.create_rmf_file(RMF_FILENAME)
rmf.set_description("Brownian dynamics trajectory with {}fs timestep.\n"\
                    .format(bd_step_size_fs))
IMP.rmf.add_hierarchy(rmf, h_root)
IMP.rmf.add_restraints(rmf, rs)
IMP.rmf.add_geometry(rmf, IMP.display.BoundingBoxGeometry(bb))
IMP.rmf.add_geometry(rmf, IMP.display.SphereGeometry(pbc_sphere))
# Pair RMF with model using an OptimizerState ("listener")
sos = IMP.rmf.SaveOptimizerState(m, rmf)
sos.set_log_level(IMP.SILENT)
sos.set_simulator(bd)
sos.set_period(rmf_dump_interval_frames)
bd.add_optimizer_state(sos)
# Dump initial frame to RMF
sos.update_always("initial conformation")

Finally, we run the simulation by calling [optimize](https://integrativemodeling.org/2.23.0/doc/ref/classIMP_1_1Optimizer.html#ab4582f19387b3ac38dbeb854e613a2c0) (this takes a minute or two to run):

In [ ]:
# -------- Run simulation ---------
print("Running simulation")
#m.update()
print("Score before: {:f}".format(sf.evaluate(True)))
bd.optimize(sim_time_frames)
print("Run finished successfully")
print("Score after: {:f}".format(sf.evaluate(True)))

# Visualization<a id="visualization"></a>

The final RMF trajectory `pbc_simple.rmf` can be viewed in UCSF Chimera, or in UCSF ChimeraX with the RMF plugin (installed from the Tools menu under 'More Tools').

# Next steps<a id="nextsteps"></a>

This demonstration uses a very simple representation of the insulin granules. See also `pbc_patches.py` and `pbc_interacting_patches.py` in the [scripts directory of the GitHub repository](https://github.com/salilab/imp_brownian_dynamics_tutorial/tree/main/scripts) for similar scripts that use a more realistic representation.